In [19]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split
from skimage.feature import local_binary_pattern
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import time
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from skimage.feature import hog


In [20]:
categories = ['jogging' , 
              'sitting' , 
              'standing' , 
              'walking']
data_directory = '/kaggle/input/data1/Dataset_images'


In [21]:
# Step 2: Data Preprocessing
image_size = (256, 256)
dataset = []
labels = []

for category in categories:
    folder_path = os.path.join(data_directory, category)
    for filename in os.listdir(folder_path):
        image_path = os.path.join(folder_path, filename)
        image = load_img(image_path, target_size=image_size)
        image_array = img_to_array(image)
        dataset.append(image_array)
        labels.append(categories.index(category))

dataset = np.array(dataset)
labels = np.array(labels)

In [8]:
hog_features = []
hog_labels = []

for image, label in zip(dataset, labels):
    
    image = cv2.cvtColor(image , cv2.COLOR_RGB2GRAY)

    # Apply HOG feature extraction
    features = hog(image, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=False,)

    # Append the features and label to the respective lists
    hog_features.append(features)
    hog_labels.append(label)

# Convert the feature and label lists to numpy arrays
hog_features = np.array(hog_features)
hog_labels = np.array(hog_labels)

In [9]:
X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(hog_features, hog_labels, test_size=0.2, random_state=42)


k = 10  
knn = KNeighborsClassifier(n_neighbors=k)

## Scaling the data
scaler = StandardScaler()
X_train_1 = scaler.fit_transform(X_train_1)
X_test_1 = scaler.transform(X_test_1)
knn.fit(X_train_1, y_train_1)

KNeighborsClassifier(n_neighbors=10)

In [22]:
folder_path = '/kaggle/input/test-images/test_images'


# Get the list of image file names in the folder
image_files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

# Initialize an empty list to store the resized images
test_images = []

# Specify the desired image size
desired_size = (256, 256)  

# Loop through the image files
for image_file in image_files:
    # Read the image using OpenCV
    image_path = os.path.join(folder_path, image_file)
    image = cv2.imread(image_path)

    # Resize the image to the desired size
    resized_image = cv2.resize(image, desired_size)

    # Append the resized image to the list
    test_images.append(resized_image)

In [15]:
# Set up a timer
start_time = time.time()

# Number of frames processed
frame_count = 0

# Inference loop
for frame in test_images:
    
    image = cv2.cvtColor(frame , cv2.COLOR_RGB2GRAY)
    
    feature  = hog(image, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2),
                          visualize=False)
    feature = feature.reshape(1, -1)
    
    pred = knn.predict(feature)
   
    frame_count += 1

# Calculate elapsed time
elapsed_time = time.time() - start_time

# Calculate FPS
fps = frame_count / elapsed_time

# Print the FPS
print("FPS: {:.2f}".format(fps))

FPS: 3.13


In [23]:
def extract_lbp_features(image):
    
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Compute LBP feature
    radius = 1  # LBP neighborhood radius
    n_points = 8 * radius  # Number of points to sample on the LBP circle
    lbp = local_binary_pattern(image, n_points, radius, method='uniform')

    # Calculate the histogram of LBP
    hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, n_points + 3), range=(0, n_points + 2))

    # Normalize the histogram
    hist = hist.astype("float")
    hist /= (hist.sum() + 1e-7)

    # Return the LBP histogram as feature vector
    return hist

In [24]:
# Initialize an empty list to store the LBP features
lbp_features = []

# Iterate over each image in the dataset and extract LBP features
for image in dataset:
    lbp_features.append(extract_lbp_features(image))

# Convert the features list to a numpy array
lbp_features = np.array(lbp_features)

/opt/conda/lib/python3.10/site-packages/skimage/feature/texture.py:353: UserWarning: Applying `local_binary_pattern` to floating-point images may give unexpected results when small numerical differences between adjacent pixels are present. It is recommended to use this function with images of integer dtype.
  warnings.warn(


In [25]:
## KNN with standard scaling

X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(lbp_features, labels, test_size=0.2, random_state=42)


k = 10  
knn = KNeighborsClassifier(n_neighbors=k)

## Scaling the data
scaler = StandardScaler()
X_train_1 = scaler.fit_transform(X_train_1)
X_test_1 = scaler.transform(X_test_1)
knn.fit(X_train_1, y_train_1)

KNeighborsClassifier(n_neighbors=10)

In [27]:
# Set up a timer
start_time = time.time()

# Number of frames processed
frame_count = 0

# Inference loop
for frame in test_images:
    
    
    feature  = extract_lbp_features(frame)
    feature = feature.reshape(1, -1)
    
    pred = knn.predict(feature)
   
    frame_count += 1

# Calculate elapsed time
elapsed_time = time.time() - start_time

# Calculate FPS
fps = frame_count / elapsed_time

# Print the FPS
print("FPS: {:.2f}".format(fps))

FPS: 35.65
